The OpenAI Gym (https://gym.openai.com) provides us with a lot of different examples and games in which to train a learning agent. The task is to develop one of such agents. We will create a neural network that, given the state of the game (actually, two consecutive states), it outputs a family of quality values (Q-values) for each next possible move. The move with higher Q-value is chosen and performed in the game. This theoretical formalism was taken from https://www.nervanasys.com/demystifying-deep-reinforcement-learning/

In [1]:
# INITIALIZATION: libraries, parameters, network...
from keras.models import Sequential      # One layer after the other
from keras.layers import Dense, Flatten  # Dense layers are fully connected layers, Flatten layers flatten out multidimensional inputs
from collections import deque            # For storing moves 
from __future__ import division

import sys
import numpy as np
sys.path.append('/home/xbucha02/libraries')
import gym                                # To train our network
#from gym import wrappers
env = gym.make('MountainCar-v0')          # Choose game (any in the gym should work)
#env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True)

import random     # For sampling batches from the observations

# Create network. Input is two consecutive game states, output is Q-values of the possible moves.
model = Sequential()  #vytvoreni linearnho modelu
model.add(Dense(20, activation="relu", kernel_initializer="uniform", input_shape=(2, 2)))
model.add(Flatten())       # Flatten input so as to have no problems with processing
model.add(Dense(18, activation="relu", kernel_initializer="uniform"))
model.add(Dense(10, activation="relu", kernel_initializer="uniform"))
model.add(Dense(3, activation="linear", kernel_initializer="uniform"))    # Same number of outputs as possible actions

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Parameters
                                # Register where the actions will be stored

observetime = 500                         # Number of timesteps we will be acting on the game and observing results
epsilon = 0.45                              # Probability of doing a random move
epsilonBackup = epsilon
gamma = 0.85                                # Discounted future reward. How much we care about steps further in time
mb_size = 50                               # Learning minibatch size
episodes = 5000                               # Pocet epizod
numberOfGames = 20
observation = env.reset()                     # Game begins

Using TensorFlow backend.
/home/xbucha02/libraries/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# FIRST STEP: Knowing what each action does (Observing)
for eps in range (episodes):
    
    
   # env.render()
    D = deque() #vyprazdneni D
    #print('{}' .format(len(D)))
    #if D:
    #    print('not gut')
    
    obs = np.expand_dims(observation, axis=0)     # (Formatting issues) Making the observation the first element of a batch of inputs 
    state = np.stack((obs, obs), axis=1)
    done = False

    for t in range(observetime):
        if np.random.rand() <= epsilon:
            action = np.random.randint(0, env.action_space.n, size=1)[0]
        else:
            Q = model.predict(state)          # Q-values predictions
            action = np.argmax(Q)             # Move with highest Q-value is the chosen one
            
        observation_new, reward, done, info = env.step(action)     # See state of the game, reward... after performing the action
        if reward != -1.0:
            print('{}. Uspech!'.format(t)) #dosazeni cile
        obs_new = np.expand_dims(observation_new, axis=0)          # (Formatting issues)
        state_new = np.append(np.expand_dims(obs_new, axis=0), state[:, :1, :], axis=1)     # Update the input with the new state of the game
        D.append((state, action, reward, state_new, done))         # 'Remember' action and consequence
        state = state_new         # Update state

        #print('{}'.format(epsilon))
        if done:
            env.reset()           # Restart game if it's finished
           # env.render()
            obs = np.expand_dims(observation, axis=0)     # (Formatting issues) Making the observation the first element of a batch of inputs 
            state = np.stack((obs, obs), axis=1)
            
    #epsilon -= (epsilonBackup - 0.1) * (1/episodes)
    #print('{}. Observing Finished' .format())

    # SECOND STEP: Learning from the observations (Experience replay)
    minibatch = random.sample(D, mb_size) #z D vybere pocet mb_size samplu

    inputs_shape = (mb_size,) + state.shape[1:]
    inputs = np.zeros(inputs_shape)
    targets = np.zeros((mb_size, env.action_space.n)) #vytvori pole ([], []), 1. argument - radky, 2. argument - sloupce

    for i in range(0, mb_size):
        state = minibatch[i][0]
        action = minibatch[i][1]
        reward = minibatch[i][2]
        state_new = minibatch[i][3]
        done = minibatch[i][4]

        # Build Bellman equation for the Q function
        inputs[i:i+1] = np.expand_dims(state, axis=0)
        targets[i] = model.predict(state)
        Q_sa = model.predict(state_new)

        if done:
            targets[i, action] = reward
        else:
            targets[i, action] = reward + gamma * np.max(Q_sa)
            #print('{}'.format(Q_sa))

    
    model.train_on_batch(inputs, targets) # Train network to output the Q function, mozna neni casti cyklu
    #print('Inputs: {}; Targets: {}'.format(inputs, targets))
    print('{}. Episode Finished'.format(eps))
    eps+= 1 #aktualizovani poctu epizod

0. Episode Finished
1. Episode Finished
2. Episode Finished
3. Episode Finished
4. Episode Finished
5. Episode Finished
6. Episode Finished
7. Episode Finished
8. Episode Finished
9. Episode Finished
10. Episode Finished
11. Episode Finished
12. Episode Finished
13. Episode Finished
14. Episode Finished
15. Episode Finished
16. Episode Finished
17. Episode Finished
18. Episode Finished
19. Episode Finished
20. Episode Finished
21. Episode Finished
22. Episode Finished
23. Episode Finished
24. Episode Finished
25. Episode Finished
26. Episode Finished
27. Episode Finished
28. Episode Finished
29. Episode Finished
30. Episode Finished
31. Episode Finished
32. Episode Finished
33. Episode Finished
34. Episode Finished
35. Episode Finished
36. Episode Finished
37. Episode Finished
38. Episode Finished
39. Episode Finished
40. Episode Finished
41. Episode Finished
42. Episode Finished
43. Episode Finished
44. Episode Finished
45. Episode Finished
46. Episode Finished
47. Episode Finished
48

378. Episode Finished
379. Episode Finished
380. Episode Finished
381. Episode Finished
382. Episode Finished
383. Episode Finished
384. Episode Finished
385. Episode Finished
386. Episode Finished
387. Episode Finished
388. Episode Finished
389. Episode Finished
390. Episode Finished
391. Episode Finished
392. Episode Finished
393. Episode Finished
394. Episode Finished
395. Episode Finished
396. Episode Finished
397. Episode Finished
398. Episode Finished
399. Episode Finished
400. Episode Finished
401. Episode Finished
402. Episode Finished
403. Episode Finished
404. Episode Finished
405. Episode Finished
406. Episode Finished
407. Episode Finished
408. Episode Finished
409. Episode Finished
410. Episode Finished
411. Episode Finished
412. Episode Finished
413. Episode Finished
414. Episode Finished
415. Episode Finished
416. Episode Finished
417. Episode Finished
418. Episode Finished
419. Episode Finished
420. Episode Finished
421. Episode Finished
422. Episode Finished
423. Episo

751. Episode Finished
752. Episode Finished
753. Episode Finished
754. Episode Finished
755. Episode Finished
756. Episode Finished
757. Episode Finished
758. Episode Finished
759. Episode Finished
760. Episode Finished
761. Episode Finished
762. Episode Finished
763. Episode Finished
764. Episode Finished
765. Episode Finished
766. Episode Finished
767. Episode Finished
768. Episode Finished
769. Episode Finished
770. Episode Finished
771. Episode Finished
772. Episode Finished
773. Episode Finished
774. Episode Finished
775. Episode Finished
776. Episode Finished
777. Episode Finished
778. Episode Finished
779. Episode Finished
780. Episode Finished
781. Episode Finished
782. Episode Finished
783. Episode Finished
784. Episode Finished
785. Episode Finished
786. Episode Finished
787. Episode Finished
788. Episode Finished
789. Episode Finished
790. Episode Finished
791. Episode Finished
792. Episode Finished
793. Episode Finished
794. Episode Finished
795. Episode Finished
796. Episo

1119. Episode Finished
1120. Episode Finished
1121. Episode Finished
1122. Episode Finished
1123. Episode Finished
1124. Episode Finished
1125. Episode Finished
1126. Episode Finished
1127. Episode Finished
1128. Episode Finished
1129. Episode Finished
1130. Episode Finished
1131. Episode Finished
1132. Episode Finished
1133. Episode Finished
1134. Episode Finished
1135. Episode Finished
1136. Episode Finished
1137. Episode Finished
1138. Episode Finished
1139. Episode Finished
1140. Episode Finished
1141. Episode Finished
1142. Episode Finished
1143. Episode Finished
1144. Episode Finished
1145. Episode Finished
1146. Episode Finished
1147. Episode Finished
1148. Episode Finished
1149. Episode Finished
1150. Episode Finished
1151. Episode Finished
1152. Episode Finished
1153. Episode Finished
1154. Episode Finished
1155. Episode Finished
1156. Episode Finished
1157. Episode Finished
1158. Episode Finished
1159. Episode Finished
1160. Episode Finished
1161. Episode Finished
1162. Episo

1476. Episode Finished
1477. Episode Finished
1478. Episode Finished
1479. Episode Finished
1480. Episode Finished
1481. Episode Finished
1482. Episode Finished
1483. Episode Finished
1484. Episode Finished
1485. Episode Finished
1486. Episode Finished
1487. Episode Finished
1488. Episode Finished
1489. Episode Finished
1490. Episode Finished
1491. Episode Finished
1492. Episode Finished
1493. Episode Finished
1494. Episode Finished
1495. Episode Finished
1496. Episode Finished
1497. Episode Finished
1498. Episode Finished
1499. Episode Finished
1500. Episode Finished
1501. Episode Finished
1502. Episode Finished
1503. Episode Finished
1504. Episode Finished
1505. Episode Finished
1506. Episode Finished
1507. Episode Finished
1508. Episode Finished
1509. Episode Finished
1510. Episode Finished
1511. Episode Finished
1512. Episode Finished
1513. Episode Finished
1514. Episode Finished
1515. Episode Finished
1516. Episode Finished
1517. Episode Finished
1518. Episode Finished
1519. Episo

...

3261. Episode Finished
3262. Episode Finished
3263. Episode Finished
3264. Episode Finished
3265. Episode Finished
3266. Episode Finished
3267. Episode Finished
3268. Episode Finished
3269. Episode Finished
3270. Episode Finished
3271. Episode Finished
3272. Episode Finished
3273. Episode Finished
3274. Episode Finished
3275. Episode Finished
3276. Episode Finished
3277. Episode Finished
3278. Episode Finished
3279. Episode Finished
3280. Episode Finished
3281. Episode Finished
3282. Episode Finished
3283. Episode Finished
3284. Episode Finished
3285. Episode Finished
3286. Episode Finished
3287. Episode Finished
3288. Episode Finished
3289. Episode Finished
3290. Episode Finished
3291. Episode Finished
3292. Episode Finished
3293. Episode Finished
3294. Episode Finished
3295. Episode Finished
3296. Episode Finished
3297. Episode Finished
3298. Episode Finished
3299. Episode Finished
3300. Episode Finished
3301. Episode Finished
3302. Episode Finished
3303. Episode Finished
3304. Episo

3618. Episode Finished
3619. Episode Finished
3620. Episode Finished
3621. Episode Finished
3622. Episode Finished
3623. Episode Finished
3624. Episode Finished
3625. Episode Finished
3626. Episode Finished
3627. Episode Finished
3628. Episode Finished
3629. Episode Finished
3630. Episode Finished
3631. Episode Finished
3632. Episode Finished
3633. Episode Finished
3634. Episode Finished
3635. Episode Finished
3636. Episode Finished
3637. Episode Finished
3638. Episode Finished
3639. Episode Finished
3640. Episode Finished
3641. Episode Finished
3642. Episode Finished
3643. Episode Finished
3644. Episode Finished
3645. Episode Finished
3646. Episode Finished
3647. Episode Finished
3648. Episode Finished
3649. Episode Finished
3650. Episode Finished
3651. Episode Finished
3652. Episode Finished
3653. Episode Finished
3654. Episode Finished
3655. Episode Finished
3656. Episode Finished
3657. Episode Finished
3658. Episode Finished
3659. Episode Finished
3660. Episode Finished
3661. Episo

3975. Episode Finished
3976. Episode Finished
3977. Episode Finished
3978. Episode Finished
3979. Episode Finished
3980. Episode Finished
3981. Episode Finished
3982. Episode Finished
3983. Episode Finished
3984. Episode Finished
3985. Episode Finished
3986. Episode Finished
3987. Episode Finished
3988. Episode Finished
3989. Episode Finished
3990. Episode Finished
3991. Episode Finished
3992. Episode Finished
3993. Episode Finished
3994. Episode Finished
3995. Episode Finished
3996. Episode Finished
3997. Episode Finished
3998. Episode Finished
3999. Episode Finished
4000. Episode Finished
4001. Episode Finished
4002. Episode Finished
4003. Episode Finished
4004. Episode Finished
4005. Episode Finished
4006. Episode Finished
4007. Episode Finished
4008. Episode Finished
4009. Episode Finished
4010. Episode Finished
4011. Episode Finished
4012. Episode Finished
4013. Episode Finished
4014. Episode Finished
4015. Episode Finished
4016. Episode Finished
4017. Episode Finished
4018. Episo

4332. Episode Finished
4333. Episode Finished
4334. Episode Finished
4335. Episode Finished
4336. Episode Finished
4337. Episode Finished
4338. Episode Finished
4339. Episode Finished
4340. Episode Finished
4341. Episode Finished
4342. Episode Finished
4343. Episode Finished
4344. Episode Finished
4345. Episode Finished
4346. Episode Finished
4347. Episode Finished
4348. Episode Finished
4349. Episode Finished
4350. Episode Finished
4351. Episode Finished
4352. Episode Finished
4353. Episode Finished
4354. Episode Finished
4355. Episode Finished
4356. Episode Finished
4357. Episode Finished
4358. Episode Finished
4359. Episode Finished
4360. Episode Finished
4361. Episode Finished
4362. Episode Finished
4363. Episode Finished
4364. Episode Finished
4365. Episode Finished
4366. Episode Finished
4367. Episode Finished
4368. Episode Finished
4369. Episode Finished
4370. Episode Finished
4371. Episode Finished
4372. Episode Finished
4373. Episode Finished
4374. Episode Finished
4375. Episo

4689. Episode Finished
4690. Episode Finished
4691. Episode Finished
4692. Episode Finished
4693. Episode Finished
4694. Episode Finished
4695. Episode Finished
4696. Episode Finished
4697. Episode Finished
4698. Episode Finished
4699. Episode Finished
4700. Episode Finished
4701. Episode Finished
4702. Episode Finished
4703. Episode Finished
4704. Episode Finished
4705. Episode Finished
4706. Episode Finished
4707. Episode Finished
4708. Episode Finished
4709. Episode Finished
4710. Episode Finished
4711. Episode Finished
4712. Episode Finished
4713. Episode Finished
4714. Episode Finished
4715. Episode Finished
4716. Episode Finished
4717. Episode Finished
4718. Episode Finished
4719. Episode Finished
4720. Episode Finished
4721. Episode Finished
4722. Episode Finished
4723. Episode Finished
4724. Episode Finished
4725. Episode Finished
4726. Episode Finished
4727. Episode Finished
4728. Episode Finished
4729. Episode Finished
4730. Episode Finished
4731. Episode Finished
4732. Episo

In [3]:
# THIRD STEP: Play!
for x in range(0, numberOfGames):
    
    obs = np.expand_dims(observation, axis=0)
    state = np.stack((obs, obs), axis=1)
    done = False
    tot_reward = 0.0
    while not done:
      #  env.render()                    # Uncomment to see game running
        #print('State: {}'.format(state))
        Q = model.predict(state)
        #print('Q: {}'.format(Q))
        action = np.argmax(Q)  
        print('Action: {}'.format(action))
        observation, reward, done, info = env.step(action)
        obs = np.expand_dims(observation, axis=0)
        state = np.append(np.expand_dims(obs, axis=0), state[:, :1, :], axis=1)    
        tot_reward += reward
    observation = env.reset()
    print('{}. Game ended! Total reward: {}'.format(x, tot_reward))

Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1


Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1


Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
8. Game ended! Total reward: -200.0
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Acti

Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
12. Game ended! Total reward: -200.0
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Act

Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
Action: 1
